In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.datasets import mnist
from keras.utils import to_categorical
import numpy as np 
import pandas as pd

# 인공신경망

## 인공신경망 (이진) 분류

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

# 1) 데이터 로드
sales_df = pd.read_csv('../../data/sales_scaled.csv')
X_train, X_test, y_train, y_test=train_test_split(
  sales_df.drop('flag',axis=1), ## 목표변수 제외한 나머지
  sales_df['flag'], ## 목표변수
  test_size=0.2, ## 테스트셋 20%
  random_state=123
)

# 2) 모델 성능 지표 출력 함수 정의
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score , recall_score, accuracy_score

def get_clf_eval(y_test,pred):
    confusion = confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    recall = recall_score(y_test,pred)
    precision = precision_score(y_test,pred)
    print('confusion matrix')
    print(confusion)
    print("정확도: {0:.4f}. 재현율: {1:.4f}, 정밀도: {2:.4f}".format(accuracy, recall, precision))

# 3) Sklearn 기반의 MLP(다층 퍼셉트론)
## 3-1) 모델 생성 및 학습
mlp_clf = MLPClassifier(
  hidden_layer_sizes=(10,10), ## 은닉층 2개, 각각 10개의 노드
  max_iter=300 ## 최대 300번 반복
)
mlp_clf.fit(X_train , y_train)

## 3-2) 예측 및 성능 지표 출력
mlp_pred = mlp_clf.predict(X_test)
get_clf_eval(y_test,mlp_pred)


# 4) Keras 기반의 MLP(다층 퍼셉트론)
from keras import models
from keras.layers import Input, Dense

## 4-1) 모델 정의
model=models.Sequential()
model.add(Input(shape=(X_train.shape[1],))) ## 입력층
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1,activation='sigmoid'))
print(model.summary())

## 4-2) 모델 컴파일 및 학습
model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
history=model.fit(
    X_train,y_train,
    epochs=200,batch_size=128,
    validation_split=0.3
)

## 4-3) 학습 결과 확인 및 평가 & 시각화
history_dict=history.history
print(history_dict.keys())
  ## dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

acc=history.history['accuracy'] ## 학습셋 정확도
val_acc=history.history['val_accuracy'] ## 검증셋 정확도
loss=history.history['loss'] ## 학습셋 손실
val_loss=history.history['val_loss'] ## 검증셋 손실
epochs=range(1,len(acc)+1) ## x축

import matplotlib.pyplot as plt 

### Loss 그래프
plt.plot(epochs,loss,'bo',label='Training loss')
plt.plot(epochs,val_loss,'b',label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.legend()
plt.show()

### Accuracy 그래프
plt.plot(epochs,acc,'bo',label='Training accuracy')
plt.plot(epochs,val_acc,'b',label='Validation accuracy')
plt.title('Training and validation Accuracy')
plt.xlabel('Epochs')
plt.legend()
plt.show()

### 4-4) 성능 지표 출력
predicted_result=model.predict(X_test)
predicted_target=pd.Series([
    1 if predicted_result[i]> 0.5 else 0 for i in range(0,predicted_result.shape[0])
    # 0.5를 기준으로 1과 0으로 변환
])
get_clf_eval(y_test,predicted_target)


## 인공신경망 손글씨 인식

In [ ]:
# 1) 데이터 로드
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train.shape ## (60000, 28, 28)

# 2) 데이터 전처리
## 3차원 데이터를 2차원으로 변환
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

## 정규화 (0~1 사이 값으로 변환)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## 목표변수 One-Hot 인코딩
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# 3) 모델 정의
model = Sequential()
model.add(Input(shape=(784,)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))
print(model.summary())

# 4) 모델 컴파일 및 학습
epochs = 10
batch_size = 128

model.compile(
  loss='categorical_crossentropy',
  metrics=['accuracy'],
  optimizer='adam'
)
history = model.fit(
  x_train, y_train,
  batch_size=batch_size,
  epochs=epochs,
  verbose=1, # 학습 과정 출력 여부
  validation_split=0.3
)

# 5) 학습 결과 확인 및 평가 & 시각화
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epoch_range = range(1, len(acc) + 1) ## x축

## Loss 그래프
plt.plot(epoch_range, loss, 'bo', label='Training loss')
plt.plot(epoch_range, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.figure()
plt.show()

## Accuracy 그래프
plt.plot(epoch_range, acc, 'bo', label='Training acc')
plt.plot(epoch_range, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.figure()

# CNN (딥러닝)

In [ ]:
# 1) 데이터 로드 및 전처리

## 데이터셋 로드 (Training, Test 데이터셋 분할)
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(X_train.shape)  # (60000, 28, 28) 이미지수, 높이, 너비

## CNN 입력을 위한 Reshape
  ### CNN 모델은 4차원 입력을 받음 (이미지수, 높이, 너비, 채널수)
  ### 흑백 이미지의 채널수는 1, 컬러 이미지의 채널수는 3
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

## 독립변수 정규화 (0~255 -> 0~1)
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

## 목표변수 One-Hot Encoding 처리
Y_train = to_categorical(y_train, 10)
Y_test = to_categorical(y_test, 10)

# 2) 모델 생성
## CNN 모델 생성
model = Sequential([
  Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),  # 첫 번째 Conv 레이어, 입력 형태 지정
  Conv2D(32, (3, 3), activation='relu'),  # 두 번째 Conv 레이어
  MaxPooling2D(pool_size=(2, 2)),  # MaxPooling 레이어로 차원 축소
  Dropout(0.25),  # Dropout 레이어로 과적합 방지
  Flatten(),  # 1차원으로 변환
  Dense(128, activation='relu'),  # Fully Connected 레이어
  Dropout(0.5),  # Dropout 레이어로 과적합 방지
  Dense(10, activation='softmax')  # 출력 레이어, 10개의 클래스
])

## 모델 요약 출력
print(model.summary())

# 3) 모델 컴파일 및 학습
## 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## 모델 학습
history = model.fit(X_train, Y_train, batch_size=32, epochs=10, verbose=1)

# 4) 모델 평가
score = model.evaluate(X_test, Y_test, verbose=0)
print(model.metrics_names)
print(score)

# 5) 성능 지표 시각화

## 성능 지표 시각화
acc = history.history['accuracy']
loss = history.history['loss']
epoch_range = range(1, len(acc) + 1)

## Loss 시각화
plt.plot(epoch_range, loss, 'b', label='Training loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.figure()

## Accuracy 시각화
plt.plot(epoch_range, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.figure()


In [ ]:
# 1) 데이터 로드
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print(X_train.shape)
  ## (50000, 32, 32, 3) 이미지수, 높이, 너비, 채널수

# 2) 데이터 전처리
## CNN 모델은 4차원 입력을 받음 (이미지수, 높이, 너비, 채널수)
X_train = X_train.reshape(X_train.shape[0], 32, 32, 3)
X_test = X_test.reshape(X_test.shape[0], 32, 32, 3)

## 독립변수 정규화 (0~255 -> 0~1) 
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

## 목표변수 One-Hot Encoding 처리
Y_train = tf.keras.utils.to_categorical(y_train, 10) ## 10개의 클래스
Y_test = tf.keras.utils.to_categorical(y_test, 10)

# 3) 모델 생성
model = Sequential()
model.add(Conv2D(
  32, # 필터 수
  (3,3), # 필터 크기
  activation='relu', # 활성화 함수
  input_shape=(32, 32, 3) # 입력 형태
))
print(model.output_shape) # (None, 30, 30, 32)

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) # MaxPooling 레이어로 차원 축소
model.add(Dropout(0.25))

model.add(Flatten())  # 1차원으로 변환
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout 레이어로 과적합 방지
model.add(Dense(10, activation='softmax')) # 출력 레이어, 10개의 클래스
print(model.summary()) # Layer, Output Shape, Param 등 출력

# 4) 모델 컴파일 및 학습
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
history=model.fit(X_train, Y_train, batch_size=32, epochs=10, verbose=1)

# 5) 성능 평가
score = model.evaluate(X_test, Y_test, verbose=0)
print(model.metrics_names)
print(score)

# 6) 성능 지표 시각화
acc = history.history['accuracy']
loss = history.history['loss']
epoch_range = range(1, len(acc) + 1) ## epoch 범위
plt.plot(epoch_range, loss, 'b', label='Training loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.figure()

plt.plot(epoch_range, acc, 'b', label='Training acc')
plt.title('Training accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.figure()


# 결측치 처리

In [ ]:
# 1) 데이터 로드
customer_df = pd.read_csv('salesdata.csv')

# 2) 데이터 정보 확인
## 데이터 정보 출력
customer_df.info()

## 각 컬럼의 고유값 확인
for col in customer_df.columns:
    print(col, customer_df[col].unique())
    ## flag ['Y' 'N']

# 3) 결측값 확인
## 결측값이 있는 컬럼 확인 및 결측값 수 확인
customer_df.isnull().sum() ## 칼럼명 결측치 수를 시리즈로 출력

## 결측값 비율 확인
(customer_df.isnull().sum() / customer_df.shape[0]) * 100

# 4) 기초 시각화
import seaborn as sns
## countplot: 
sns.countplot(x='BikeBuyer', data=customer_df)
## histplot
sns.histplot(x='AvgMonthSpend', data=customer_df)

# 5) 결측값 처리
## 결측값이 있는 행 제거
customer_df.dropna(subset=['column명'], inplace=True) ## 특정 칼럼에 결측값이 있을 경우 해당 행을 제거

## 결측행 채우기
customer_df.fillna('값', inplace=True)

# 레이블 인코딩

**레이블 인코딩**

레이블 인코딩이란?
레이블 인코딩은 범주형 데이터 (보통 서열 척도)를 숫자로 변환하는 기법입니다. 

예를 들어, '사과', '바나나', '체리'라는 범주형 데이터가 있을 때, 
이를 각각 0, 1, 2로 변환할 수 있습니다. (영어 > 한글 사전순)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# 1) 데이터 로드
items=['TV','냉장고','전자렌지','컴퓨터','선풍기','선풍기','믹서','믹서']

# 2) 인코딩 변환
encoder = LabelEncoder() ## 인코더 초기화
encoder.fit(items) ## 인코더 학습
labels = encoder.transform(items) ## 문자열 -> 인코딩된 숫자로 변환
print('인코딩 변환값:',labels) # [0 1 4 5 3 3 2 2]

# 3) 인코딩 클래스 (고유값) 확인
print('인코딩 클래스:',encoder.classes_) # ['TV' '냉장고' '믹서' '선풍기' '전자렌지' '컴퓨터']

# 4) 디코딩
print('디코딩 원본 값:',encoder.inverse_transform([4, 5, 2, 0, 1, 1, 3, 3]))

In [ ]:
# 수동 인코딩 (df.replace)
sales_df = pd.read_csv('sales_na.csv')
sales_df['flag'].replace(["N","Y"],[0,1],inplace=True)

# 데이터프레임의 각 칼럼을 라벨 인코딩
le = LabelEncoder()
for col in sales_df.columns:
  le.fit(customer_df[col])
  customer_df[col] = le.transform(customer_df[col])

# 원 핫 인코딩과 Min-Max 스케일링

## 원 핫 인코딩(One-Hot Encoding)

원 핫 인코딩(One-Hot Encoding)은 범주형 데이터 (주로 명목 척도)를 수치형 데이터로 변환하는 기법 중 하나입니다.
이 방법은 각 범주를 고유한 이진 벡터로 변환합니다.
예를 들어, '빨강', '초록', '파랑'이라는 세 가지 범주가 있을 때,
이를 원 핫 인코딩하면 다음과 같이 변환됩니다:

빨강: [1, 0, 0]
초록: [0, 1, 0]
파랑: [0, 0, 1]

In [1]:
import pandas as pd

# 1) 데이터 로드
df = pd.DataFrame({'item':['TV','냉장고','전자렌지','컴퓨터','선풍기','선풍기','믹서','믹서'] })
print(df.columns)

# 2) 원핫 인코딩 - 특정 칼럼을 원핫 인코딩한 칼럼들로 대체
one_hot_result = pd.get_dummies(df, columns=['item'])
  ## 해당 칼럼을 원핫인코딩으로 변환 (기존 칼럼은 제거)
print(one_hot_result.columns)

Index(['item'], dtype='object')
Index(['item_TV', 'item_냉장고', 'item_믹서', 'item_선풍기', 'item_전자렌지', 'item_컴퓨터'], dtype='object')


In [9]:
# customer_df 데이터셋

# 1) 데이터 로드
customer_df = pd.read_csv('../../data/customer_na.csv')

# 2) 원핫 인코딩
## replace 인코딩
customer_df['Gender'] = customer_df['Gender'].replace(['M', 'F'], [0, 1])
customer_df['MaritalStatus'] = customer_df['MaritalStatus'].replace(['S', 'M'], [0, 1])

## get_dummies 인코딩
columns = ['CountryRegionName', 'Education', 'Occupation']
customer_df = pd.get_dummies(customer_df, columns=columns)

## Min-Max 스케일링 (Min-Max Scaling)

Min-Max 스케일링은 데이터의 값을 0과 1 사이의 값으로 변환하는 정규화 기법입니다.
```
수식: X_scaled = (X - X_min) / (X_max - X_min)
```


이 외에 정규화, Feature Scaling 등이 있습니다.

In [19]:
from sklearn.preprocessing import MinMaxScaler

# 0) Min-Max 스케일러 초기화
scaler = MinMaxScaler()

# 1) 전처리: int -> float
customer_df = customer_df.astype('float')

# 2) Min-Max 스케일러 훈련 - Min과 Max 계산
scaler.fit(customer_df)

# 3) 데이터 변환 (numpy.ndarray로 반환)
customer_df_scaled_array = scaler.transform(customer_df)

# 4) 변환된 데이터를 DataFrame으로 변환
scaler_df_scaled = pd.DataFrame(customer_df_scaled_array, columns=customer_df.columns)

# 5) 결과 출력
print(scaler_df_scaled)

       Gender  MaritalStatus  HomeOwnerFlag  NumberCarsOwned  \
0         0.0            1.0            1.0              0.6   
1         0.0            1.0            1.0              0.4   
2         1.0            0.0            0.0              0.6   
3         0.0            1.0            1.0              0.4   
4         0.0            0.0            1.0              0.2   
...       ...            ...            ...              ...   
18350     1.0            1.0            1.0              0.2   
18351     1.0            0.0            0.0              0.2   
18352     1.0            0.0            0.0              0.0   
18353     1.0            0.0            0.0              0.2   
18354     1.0            1.0            1.0              0.0   

       NumberChildrenAtHome  TotalChildren  YearlyIncome  BikeBuyer  \
0                  0.000000       0.333333      0.496842        1.0   
1                  0.333333       0.666667      0.489453        1.0   
2                 

# K-means 군집화

## K-means 군집화 및 시각화 (+PCA)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.datasets import load_iris


# 1) 데이터 로드
iris = load_iris()
irisDF = pd.DataFrame(data=iris.data, columns=[
  'sepal_length','sepal_width','petal_length','petal_width'
])

# 2) KMeans 모델 학습
kmeans = KMeans(
  n_clusters=3, ## 군집 개수
  init='k-means++', ## 초기 중심점 설정 방식
  max_iter=300, ## 최대 반복 횟수
  random_state=0 ## 시드값
).fit(irisDF)

# 3) 군집 결과 확인
kmeans.labels_ # 각 데이터가 포인트가 속한 군집 출력
  ## [1 1 0 1 2 1 0 ...] 군집 0, 1, 2 중 하나로 분류
kmeans.cluster_centers_ # 각 군집 중심점 좌표 (4차원)
  ## [[5.9016129  2.7483871  4.39354839 1.43387097], [...], [...]]

# 4) 군집 결과를 데이터프레임에 추가
irisDF['cluster']=kmeans.labels_ # 군집 결과를 데이터프레임에 추가
irisDF['target'] = iris.target # 실제 품종 (정답)

# 5) 차원 축소: 4차원 데이터를 2차원으로
from sklearn.decomposition import PCA

## 5-1) PCA 모델 생성
pca = PCA(n_components=2) ## n_components: 축소할 차원 수

## 5-2) PCA 모델 학습
pca_transformed = pca.fit_transform(iris.data)
  ## 150개의 데이터가 [x좌표, y좌표] 형태의 2차원 데이터로 변환
  ## array([[2.68420713,  0.32660731], ...]),

## 5-3) PCA 데이터를 데이터프레임에 추가
irisDF['pca_x'] = pca_transformed[:,0]
irisDF['pca_y'] = pca_transformed[:,1]

# 6) 군집 시각화

## 6-1) Scatter plot (cluster 값이 0, 1, 2 인 경우마다 별도의 Index로 추출)
marker0_ind = irisDF[irisDF['cluster']==0].index
marker1_ind = irisDF[irisDF['cluster']==1].index
marker2_ind = irisDF[irisDF['cluster']==2].index

plt.scatter(
  x=irisDF.loc[marker0_ind,'pca_x'], ## cluster값 0, 1, 2에 해당하는 Index로, 각 cluster 레벨의 pca_x, pca_y 값 추출
  y=irisDF.loc[marker0_ind,'pca_y'], 
  marker='o') ## o, s, ^ 로 marker 표시
plt.scatter(x=irisDF.loc[marker1_ind,'pca_x'], y=irisDF.loc[marker1_ind,'pca_y'], marker='s') # 사각형
plt.scatter(x=irisDF.loc[marker2_ind,'pca_x'], y=irisDF.loc[marker2_ind,'pca_y'], marker='^') # 삼각형
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('3 Clusters Visualization by 2 PCA Components')
plt.show()

## 6-2) boxplot
import seaborn as sns
plt.figure(figsize=(6,6))
g=sns.boxplot(data=irisDF, y='sepal_length',x='cluster')
g.set_title('Sepal Length per Cluster')
g.set_xlabel('Cluster')
g.set_ylabel('Sepal Length')


In [ ]:
# 1) 데이터 로드
boston_df = pd.read_csv('boston_housing.csv')

# 2) Min-Max 스케일링
boston_scaled = MinMaxScaler().fit_transform(boston_df.drop('medv',axis=1))
boston_scaled_df=pd.DataFrame(data=boston_scaled,columns=boston_df.columns[:-1])

# 3) K-means 군집화
kmeans = KMeans(n_clusters=7, init='k-means++', max_iter=300,random_state=0).fit(boston_scaled_df)
print(kmeans.labels_)
print(kmeans.cluster_centers_)
boston_df['cluster']=kmeans.labels_

# 4) PCA 차원축소 및 시각화
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_transformed = pca.fit_transform(boston_scaled_df)
boston_df['pca_x'] = pca_transformed[:,0]
boston_df['pca_y'] = pca_transformed[:,1]

## 군집의 개수(7개)만큼 산점도 시각화
marker0_ind = boston_df[boston_df['cluster']==1].index
marker1_ind = boston_df[boston_df['cluster']==1].index
marker2_ind = boston_df[boston_df['cluster']==2].index
marker3_ind = boston_df[boston_df['cluster']==3].index
marker4_ind = boston_df[boston_df['cluster']==4].index
marker5_ind = boston_df[boston_df['cluster']==5].index
marker6_ind = boston_df[boston_df['cluster']==6].index
plt.scatter(x=boston_df.loc[marker0_ind,'pca_x'], y=boston_df.loc[marker0_ind,'pca_y'], marker='o') 
plt.scatter(x=boston_df.loc[marker1_ind,'pca_x'], y=boston_df.loc[marker1_ind,'pca_y'], marker='s')
plt.scatter(x=boston_df.loc[marker2_ind,'pca_x'], y=boston_df.loc[marker2_ind,'pca_y'], marker='^')
plt.scatter(x=boston_df.loc[marker3_ind,'pca_x'], y=boston_df.loc[marker3_ind,'pca_y'], marker='v') 
plt.scatter(x=boston_df.loc[marker4_ind,'pca_x'], y=boston_df.loc[marker4_ind,'pca_y'], marker='<')
plt.scatter(x=boston_df.loc[marker5_ind,'pca_x'], y=boston_df.loc[marker5_ind,'pca_y'], marker='>')
plt.scatter(x=boston_df.loc[marker6_ind,'pca_x'], y=boston_df.loc[marker6_ind,'pca_y'], marker='*')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('7 Clusters Visualization by 2 PCA Components')
plt.show()

# 5) 군집별 특징 비교 & 시각화
## 주태가격 평균 비교
boston_df.groupby(['cluster'])['medv'].mean()

## 중간값 표시를 위한 boxplot
import seaborn as sns
g=sns.boxplot(data=boston_df,y='medv',x='cluster')
g.set_title('medv per Cluster')
g.set_xlabel('Cluster')
g.set_ylabel('medv')

## 중간값이 가장 높은 / 낮은 군집의 특징 확인
boston_df[boston_df['cluster']==4].describe()

## 군집별 범죄율(crim)을 boxplot으로 시각화
g=sns.boxplot(data=boston_df,y='crim',x='cluster')
g.set_title('crim per Cluster')
g.set_xlabel('Cluster')
g.set_ylabel('crim')


## 클러스터링 알고리즘 테스트

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

# 1) 랜덤한 데이터 생성 (for 군집화 알고리즘 테스트)
X, y = make_blobs(n_samples=200, n_features=2, centers=3, cluster_std=0.8, random_state=0)
  # n_samples : 생성할 총 데이터 개수
  # n_features : 데이터의 피처 개수 (차원)
  # centers : 군집의 개수
  # cluster_std : 군집 내에서의 표준 편차
unique, counts = np.unique(y, return_counts=True) ## 각 군집의 데이터 개수 확인
print(unique, counts)
  ## [0 1 2] [67 67 66]

# 2) 데이터 프레임으로 변환
clusterDF = pd.DataFrame(data=X, columns=['ftr1', 'ftr2'])
clusterDF['target'] = y

# 3) 데이터 시각화: Scatter (군집화 전)
target_list = np.unique(y) # 군집의 종류: [0 1 2]
markers=['o', 's', '^', 'P','D','H','x']
for target in target_list:
    target_cluster = clusterDF[clusterDF['target']==target]
    plt.scatter(x=target_cluster['ftr1'], y=target_cluster['ftr2'], edgecolor='k', marker=markers[target] )
plt.show()

# 4) KMeans 군집화
kmeans = KMeans(
    n_clusters=3,
    init='k-means++',
    max_iter=200,
    random_state=0
)

## 군집화 결과(cluster_labels)를 저장
cluster_labels = kmeans.fit_predict(X)
clusterDF['kmeans_label'] = cluster_labels
unique_labels = np.unique(cluster_labels)


# 5) 군집화 결과 시각화
## 군집 중심 좌표
centers = kmeans.cluster_centers_
unique_labels = np.unique(cluster_labels)

## marker
markers=['o', 's', '^', 'P','D','H','x']

## Scatter plot
for label in unique_labels:
    
    # 군집 시각화
    label_cluster = clusterDF[clusterDF['kmeans_label']==label]
    plt.scatter(x=label_cluster['ftr1'], y=label_cluster['ftr2'], edgecolor='k', 
                marker=markers[label] )
    
    # 군집별 중심 위치 좌표 시각화
    center_x_y = centers[label]
    plt.scatter(x=center_x_y[0], y=center_x_y[1], s=200, color='white',
                alpha=0.9, edgecolor='k', marker=markers[label])
    plt.scatter(x=center_x_y[0], y=center_x_y[1], s=70, color='k', edgecolor='k', 
                marker='$%d$' % label)
    
plt.show()

# 가우시안 믹스처 모델 (GMM)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris, make_blobs
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

# 1) 데이터 로드
iris = load_iris()
feature_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
irisDF = pd.DataFrame(data=iris.data, columns=feature_names)
irisDF['target'] = iris.target

# 2) 클러스터링 결과 저장 (GMM, KMeans)
## 2-1) GMM 클러스터링
gmm = GaussianMixture(
  n_components=3, # 군집 개수
  random_state=0
).fit(iris.data)
irisDF['gmm_cluster'] = gmm.predict(iris.data)

## 2-2) KMeans 클러스터링
kmeans = KMeans(n_clusters=3, random_state=0).fit(iris.data)
irisDF['kmeans_cluster'] = kmeans.labels_

# 3) 결과 확인
## 3-1) GMM
print("\nGMM Clustering Results:")
print(irisDF.groupby('target')['gmm_cluster'].value_counts())

## 3-2) KMeans
print("KMeans Clustering Results:")
print(irisDF.groupby('target')['kmeans_cluster'].value_counts())

In [ ]:
# 0) 시각화 함수 정의
def visualize_cluster_plot(clusterobj, dataframe, label_name, iscenter=True):
  if iscenter: # 중심 좌표가 있는 경우
    centers = clusterobj.cluster_centers_
  
  unique_labels = np.unique(dataframe[label_name].values)
  markers = ['o', 's', '^', 'x', '*']
  isNoise = False

  for label in unique_labels:
    label_cluster = dataframe[dataframe[label_name] == label]
    if label == -1: ## label이 -1인 경우, Noise로 분류
      cluster_legend = 'Noise'
      isNoise = True
    else:
      cluster_legend = 'Cluster ' + str(label) # 군집 이름
    
    # 군집별 시각화
    plt.scatter(x=label_cluster['ftr1'], y=label_cluster['ftr2'], s=70,
          edgecolor='k', marker=markers[label], label=cluster_legend)
    
    # 군집 중심점 시각화
    if iscenter:
      center_x_y = centers[label]
      plt.scatter(x=center_x_y[0], y=center_x_y[1], s=250, color='white',
            alpha=0.9, edgecolor='k', marker=markers[label])
      plt.scatter(x=center_x_y[0], y=center_x_y[1], s=70, color='k',
            edgecolor='k', marker='$%d$' % label)
  if isNoise:
    legend_loc = 'upper center' # Noise의 경우, legend(범례) 위치를 달리하여 표시
  else:
    legend_loc = 'upper right' # Noise가 아닌 경우, 기본 위치
  
  plt.legend(loc=legend_loc)
  plt.show()

# 1) 데이터 생성
X, y = make_blobs(n_samples=300, n_features=2, centers=3, cluster_std=0.5, random_state=0)
clusterDF = pd.DataFrame(data=X, columns=['ftr1', 'ftr2'])
clusterDF['target'] = y

# 2) 데이터 시각화 (탐색)
visualize_cluster_plot(None, clusterDF, 'target', iscenter=False)

## 타원형 데이터셋으로 변환
transformation = [[0.60834549, -0.63667341], [-0.40887718, 0.85253229]]
X_aniso = np.dot(X, transformation) # Linear Transformation
clusterDF = pd.DataFrame(data=X_aniso, columns=['ftr1', 'ftr2'])
clusterDF['target'] = y
visualize_cluster_plot(None, clusterDF, 'target', iscenter=False)

# 3) GMM, KMeans 클러스터링 결과 비교
## 3-1) GMM 클러스터링 시각화
gmm = GaussianMixture(n_components=3, random_state=0)
clusterDF['gmm_label'] = gmm.fit(X_aniso).predict(X_aniso)

print('\n### Gaussian Mixture Clustering ###')
print(clusterDF.groupby('target')['gmm_label'].value_counts())
visualize_cluster_plot(gmm, clusterDF, 'gmm_label', iscenter=False)

## 3-2) KMeans 클러스터링 시각화
kmeans = KMeans(n_clusters=3, random_state=0)
kmeans_label = kmeans.fit_predict(X_aniso)
clusterDF['kmeans_label'] = kmeans_label
  ## 또는 clusterDF['kmeans_label'] = kmeans.labels_

print('### KMeans Clustering ###')
print(clusterDF.groupby('target')['kmeans_label'].value_counts())
visualize_cluster_plot(kmeans, clusterDF, 'kmeans_label', iscenter=True)

In [ ]:
# 1) 데이터 로드
boston_df = pd.read_csv('../../data/boston_housing.csv')
boston_df

# 2) 데이터 전처리
## 2-1) Min-Max 스케일링
boston_scaled = MinMaxScaler().fit_transform(boston_df.drop('medv', axis=1))
boston_scaled_df=pd.DataFrame(data=boston_scaled, columns=boston_df.columns[:-1])

## 2-2) PCA
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_transformed = pca.fit_transform(boston_scaled_df)

boston_df['ftr1'] = pca_transformed[:,0]
boston_df['ftr2'] = pca_transformed[:,1]

# 3) GMM 군집화
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components=7, random_state=0).fit(boston_scaled_df)
boston_df['cluster']=gmm.predict(boston_scaled_df)

# 4) 군집별 시각화
visualize_cluster_plot(gmm, boston_df,'cluster',iscenter=False)


# DBSCAN 군집화

In [ ]:
from sklearn.datasets import load_iris

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# 1) 데이터 로드
iris = load_iris()
feature_names = ['sepal_length','sepal_width','petal_length','petal_width']
irisDF = pd.DataFrame(data=iris.data, columns=feature_names)
irisDF['target'] = iris.target

# 2) DBSCAN 군집화 수행 및 결과 저장
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(
  eps=0.6, # 같은 군집으로 인식할 데이터포인트의 최대 거리
  min_samples=8, # 군집을 형성하기 위한 최소 데이터포인트 수
  metric='euclidean' # 거리 측정 방식
)
dbscan_labels = dbscan.fit_predict(iris.data)
irisDF['dbscan_cluster'] = dbscan_labels
irisDF['target'] = iris.target

# 3) 군집화 결과 확인
iris_result = irisDF.groupby(['target'])['dbscan_cluster'].value_counts()
print(iris_result)

# 4) 군집 시각화
## 4-1) PCA 차원 축소
from sklearn.decomposition import PCA
pca = PCA(n_components=2, random_state=0)
pca_transformed = pca.fit_transform(iris.data)
irisDF['ftr1'] = pca_transformed[:,0]
irisDF['ftr2'] = pca_transformed[:,1]

## 4-2) 군집 시각화
visualize_cluster_plot(dbscan, irisDF, 'dbscan_cluster', iscenter=False)


In [ ]:
# 1) 데이터 로드 (make_circle)
from sklearn.datasets import make_circles

X, y = make_circles(n_samples=1000, shuffle=True, noise=0.05, random_state=0, factor=0.5)
clusterDF = pd.DataFrame(data=X, columns=['ftr1', 'ftr2'])
clusterDF['target'] = y

# 2) 데이터 시각화 (탐색)
visualize_cluster_plot(None, clusterDF, 'target', iscenter=False)

# 3) Kmeans / GMM / MeanShift / DBSCAN 클러스터링 비교 (시각화)
## 3-1) Kmeans
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, max_iter=1000, random_state=0)
kmeans_labels = kmeans.fit_predict(X)
clusterDF['kmeans_cluster'] = kmeans_labels

visualize_cluster_plot(kmeans, clusterDF, 'kmeans_cluster', iscenter=True)

## 3-2) GMM
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=2, random_state=0)
gmm_label = gmm.fit(X).predict(X)
clusterDF['gmm_cluster'] = gmm_label

visualize_cluster_plot(gmm, clusterDF, 'gmm_cluster', iscenter=False)

## 3-3) MeanShift
from sklearn.cluster import MeanShift
meanshift= MeanShift(
  bandwidth=0.9 # 커널의 크기 (-> 클러스터의 크기에 영향을 미침)
)
meanshift_label = meanshift.fit_predict(X)
clusterDF['ms_cluster'] = meanshift_label

visualize_cluster_plot(meanshift, clusterDF, 'ms_cluster', iscenter=False)

## 3-4) DBSCAN
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.1, min_samples=10, metric='euclidean')
dbscan_labels = dbscan.fit_predict(X)
clusterDF['dbscan_cluster'] = dbscan_labels

visualize_cluster_plot(dbscan, clusterDF, 'dbscan_cluster', iscenter=False)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1) 데이터 로드
boston_df = pd.read_csv('boston.csv')

# 2) 데이터 전처리
## 2-1) Min-Max Scaling
scaler = MinMaxScaler()
boston_scaled = scaler.fit_transform(boston_df.drop('medv', axis=1))
boston_scaled_df = pd.DataFrame(boston_scaled, columns=boston_df.columns[:-1])

## 2-2) PCA
pca = PCA(n_components=2)
pca_transformed = pca.fit_transform(boston_scaled_df)
boston_df['ftr1'] = pca_transformed[:,0]
boston_df['ftr2'] = pca_transformed[:,1]

# 3) DBSCAN 군집화 & 시각화
dbscan = DBSCAN(eps=0.5, min_samples=5, metric='euclidean')
dbscan_labels = dbscan.fit_predict(boston_scaled_df)
boston_df['dbscan_cluster_0_5_5'] = dbscan_labels
visualize_cluster_plot(dbscan, boston_df, 'dbscan_cluster_0_5_5', iscenter=False)

## 이후에 eps, min_samples를 조정하여 군집화 결과 확인

# 연관 규칙

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# 1) 데이터 로드 (트랜잭션 데이터)
dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
       ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
       ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
       ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
       ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

# 2) 트랜잭션 인코딩
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset) ## 이진(Binary) 인코딩
       ## array([[False False  True False  True  True  True  True  True], [...]])
df = pd.DataFrame(te_ary, columns=te.columns_)

# 3) ItemSet 생성
frequent_itemsets = apriori( ## apriori: 빈번한 아이템 집합을 찾는 알고리즘
       df, # 인코딩된 데이터
       min_support=0.6, # 최소 지지도
       use_colnames=True # 아이템명 사용
)

# 4) 연관 규칙 생성
rules = association_rules(
       frequent_itemsets, # ItemSet
       metric="lift", # 규칙 평가 척도 (lift || confidence)
       min_threshold=1.2 # 기준(metric)의 최솟값
)
#      antecedents(조건)  consequents(결과)  support  confidence   lift
# 0         (Milk)        (Kidney Beans)    0.6        0.75    1.25
# 1         (Eggs)        (Kidney Beans)    0.8        0.89    1.35


# 5) 연관규칙 결과 조회
rules
## 필터링 (조건에 맞는 연관규칙 조회)
rules[ (rules['antecedent_len'] >= 2) &
       (rules['confidence'] > 0.75) &
       (rules['lift'] > 1.2) ]
rules[rules['antecedents'] == {'Eggs', 'Kidney Beans'}]
rules[rules['antecedents'].apply(lambda x: 'Eggs' in x)]

rules.iloc[rules['lift'].idxmax()] # lift가 가장 큰 규칙

# Top 10 규칙
rules.sort_values('confidence',ascending=False)[['antecedents','consequents','support','confidence','lift']].head(10)

# 